In [1]:
import re
import string
import codecs
import pickle
import pandas as pd
import numpy as np
from collections import Counter

from pythainlp.tokenize import word_tokenize
from pythainlp import word_vector, Tokenizer, sent_tokenize
from pythainlp.util import dict_trie, normalize
from pythainlp.ulmfit import *
from pythainlp.corpus.common import thai_words
from pythainlp.corpus import thai_stopwords, wordnet

from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
df_p = pd.read_csv('1.csv').applymap(str) #position name
df_d = pd.read_csv('2544_text.csv').rename(columns={"Position":"Description"}).applymap(str) #description

df = pd.concat([df_d,df_p.rename(columns={'Position':'Description'})],axis=0) #cancat df_d and df_p

In [3]:
y_train = df_d["Classification"]

tfidf = TfidfVectorizer( ngram_range=(1,2),tokenizer=process_thai,                             
                        sublinear_tf=True,
                        )
tfidf_fit = tfidf.fit(df["Description"])
X_train = tfidf_fit.transform(df_d["Description"])



In [4]:
lgt = LogisticRegression(C=2., penalty="l2", solver="liblinear", dual=False, multi_class="ovr")
lgt.fit(X_train,y_train)
lgt.score(X_train,y_train)

0.9469604243166054

In [5]:
dcs = DecisionTreeClassifier()
dcs.fit(X_train,y_train)
dcs.score(X_train,y_train)

0.9995920032639739

In [7]:
rdmf = RandomForestClassifier()
rdmf.fit(X_train, y_train)
rdmf.score(X_train, y_train)

0.9995920032639739

In [10]:
def position(text):
    doc_sample = [str(text)]
    z = tfidf_fit.transform(doc_sample)
    dcs_p = df_p['Position'][df_p['Classification']==dcs.predict(z)[0]].tolist()
    lgt_p = df_p['Position'][df_p['Classification']==lgt.predict(z)[0]].tolist()
    rdmf_p = df_p['Position'][df_p['Classification']==rdmf.predict(z)[0]].tolist()
    print('DecisionTree\n',dcs_p,'\n','Logistic\n',lgt_p,'\n','RandomForest\n',rdmf_p)

In [12]:
position('ทำความสะอาดบ้าน')

DecisionTree
 [] 
 Logistic
 ['นักแสดง ผู้กำกับภาพยนต์ ละคร และงานอื่น ๆ ที่เกี่ยวข้อง', 'นักแสดง', 'ละครวิทยุ', 'นักเล่านิทาน', 'ผู้แสดงแทนนักแสดง', 'ผู้คัดเลือกนักแสดง , เจ้าหน้าที่จัดหานักแสดง', 'ผู้กำกับการแสดงภาพยนตร์', 'ผู้กำกับการแสดงละครโทรทัศน์', 'ผู้กำกับการแสดง (ละครเวที)', 'ผู้กำกับภาพ (ภาพยนตร์)', 'ผู้กำกับภาพ (โทรทัศน์)', 'ผู้กำกับเสียง (ภาพยนตร์)', 'ผู้กำกับเสียง (โทรทัศน์)', 'ผู้กำกับเทคนิค (โทรทัศน์)', 'ผู้กำกับแสง (โทรทัศน์)', 'ผู้ถ่ายภาพนิ่ง (ภาพยนตร์)', 'ผู้ควบคุมการผลิตรายการโทรทัศน์', 'ผู้จัดรายการโทรทัศน์', 'ผู้ควบคุมรายการโทรทัศน์', 'ผู้กำกับรายการโทรทัศน์', 'เจ้าหน้าที่สถานที่รายการละครโทรทัศน์', 'เจ้าหน้าที่จัดผังรายการวิทยุกระจายเสียง , เจ้าหน้าที่ฝ่ายรายการ', 'เจ้าหน้าที่ข้อมูลรายการโทรทัศน์', 'เจ้าหน้าที่ข้อมูลรายการภาพยนตร์', 'ผู้ประสานงานรายการและงานข่าว', 'ผู้กำกับศิลปกรรม (โทรทัศน์และภาพยนตร์)', 'เจ้าหน้าที่งานฉากและศิลปกรรมรายการละครโทรทัศน์', 'ผู้กำกับเวที (โทรทัศน์)', 'ผู้กำกับเวที (ละครเวที)', 'ผู้กำกับศิลป์ (งานโฆษณา)', 'เจ้าหน้าที่สร้างสรรค์รายการ

In [ ]:
# with open('tfidf_fit.pkl', 'wb') as file:
#     pickle.dump(tfidf_fit, file)
# with open('dcs_model.pkl', 'wb') as file:
#     pickle.dump(dcs, file)
# with open('lgt_model.pkl', 'wb') as file:
#     pickle.dump(lgt, file)
# with open('rdmf_model.pkl', 'wb') as file:
#     pickle.dump(rdmf, file)